In [2]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re

import acquire

# Lesson

In [2]:
url = 'https://web-scraping-demo.zgulde.net/people'
response = get(url)

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
people = soup.select('div.person')

In [6]:
person = people[0]

In [7]:
dct = {}
dct['name'] = person.find('h2').text
dct['quote'] = person.find('p').text
dct['email'] = person.select('div')[0].select('p')[0].text
dct['phone'] = person.select('div')[0].select('p')[1].text
dct['address'] = person.select('div')[1].select('p')[0].text

In [8]:
def parse_person(person):
    dct = {}
    dct['first_name'] = person.select_one('.name').text.split()[0].strip()
    dct['last_name'] = person.select_one('.name').text.split()[1].strip()
    dct['quote'] = person.select_one('.quote').text.strip()
    dct['email'] = person.select_one('.email').text.strip()
    dct['phone'] = person.select_one('.phone').text.strip()
    dct['address'] = person.select_one('.address').text.strip()
    return dct

In [9]:
df = pd.DataFrame([parse_person(person) for person in people])
df['street'] = df.address.str.extract(r'^(.+)\n')
df.head()

,first_name,last_name,quote,email,phone,address,street
0,Erika,Singh,"""Expanded upward-trending migration""",heather10@sanford-tyler.com,(720)024-8715,86428 Carpenter Hills \n West A...,86428 Carpenter Hills
1,David,Hill,"""Future-proofed coherent access""",beckmaureen@yahoo.com,+1-335-855-9267,853 Barrett Passage \n Victoria...,853 Barrett Passage
2,Brittany,Robertson,"""Networked fresh-thinking contingency""",curtisdennis@gmail.com,(278)228-4260x73351,396 Jennifer Trail \n West Mich...,396 Jennifer Trail
3,Victoria,Jacobs,"""Stand-alone maximized intranet""",smithrobert@reyes-gonzalez.org,+1-234-910-4214x821,2374 Nicole Ramp \n Jonathanfur...,2374 Nicole Ramp
4,Christine,Davenport,"""Pre-emptive needs-based core""",tina36@hotmail.com,186-423-9531x65330,3107 Brian Estates Suite 508 \n ...,3107 Brian Estates Suite 508


# Exercises

## 1. Codeup Blog Articles

Visit Codeup's Blog and record the urls for at least 5 distinct blog posts. For each post, you should scrape at least the post's title and content.

Encapsulate your work in a function named get_blog_articles that will return a list of dictionaries, with each dictionary representing one article. The shape of each dictionary should look like this:

{
    'title': 'the title of the article',
    'content': 'the full text content of the article'
}

In [3]:
def get_blog_articles(use_cache=True):
    
    # establish a filename for the local csv
    filename = 'codeup_blog_articles.csv'
    
    if use_cache:
        
        # check to see if a local copy already exists
        if os.path.exists(filename):
            print('Reading from local CSV...')
            # if so, return the local csv
            return pd.read_csv(filename)
        
    # otherwise, scrape the data from codeup.com
    print('Reading blog articles from codeup.com...')
    
    articles = []

    # go to blog homepage
    url = 'https://codeup.com/blog/'
    headers = {'user-agent': 'Innis Data Science Cohort'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # get url for next page of articles
    # (returns None if there are no more pages)
    next_page = soup.select_one('.pagination.clearfix').div.a

    # get the urls for the rest of the articles on this page
    urls = []
    for article in soup.select('article'):
        #for link in article.select('.more-link'):
        for link in article.select('.entry-featured-image-url'):
            urls.append(link.attrs['href'])

    # go to each article page
    for url in urls:
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # pull article info and append to list
        dct = {}
        dct['title'] = soup.select_one('.entry-title').text
        dct['content'] = soup.select_one('.entry-content').text.strip()
        articles.append(dct)

    page_counter = 1
    print(f'{page_counter} pages complete     ', end='\r')

    # check whether there is a next page
    while next_page != None:
        # go to the next page
        url = next_page.attrs['href']
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # get url for next page of articles
        # (this will return None if there are no more pages)
        next_page = soup.select_one('.pagination.clearfix').div.a

        # get all the urls for articles on this page
        urls = []
        for article in soup.select('article'):
            for link in article.select('.entry-featured-image-url'):
                urls.append(link.attrs['href'])

        # go to each article page
        for url in urls:
            response = get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')

            # pull article info and append to list
            dct = {}
            dct['title'] = soup.select_one('.entry-title').text
            dct['content'] = soup.select_one('.entry-content').text.strip()
            articles.append(dct)

        page_counter += 1
        print(f'{page_counter} pages complete     ', end='\r')
        
    print(f'{page_counter} pages scraped. No more pages available.')
    
    articles = pd.DataFrame(articles)
    
    # cache local copy
    print('Writing to local CSV...')
    articles.to_csv(filename, index=False)
    
    return articles

In [4]:
df = acquire.get_blog_articles()

Reading from local CSV...


In [7]:
df

,title,content
0,From Bootcamp to Bootcamp | A Military Appreci...,"In honor of Military Appreciation Month, join ..."
1,Our Acquisition of the Rackspace Cloud Academy...,"Just about a year ago on April 16th, 2021 we a..."
2,Learn to Code: HTML & CSS on 4/30,HTML & CSS are the design building blocks of a...
3,Learn to Code: Python Workshop on 4/23,"According to LinkedIn, the “#1 Most Promising ..."
4,Coming Soon: Cloud Administration,We’re launching a new program out of San Anton...
...,...,...
193,The College Financing Scam and How We’re Doing...,Here’s the dirty secret of private college “fi...
194,Why People Can’t Learn Programming on Their Own,"While developing Codeup, we interviewed dozens..."
195,What is Our Noble Cause?,In his TEDx San Antonio presentation this fall...
196,Scholarships for Women: Why We’re Doing It,A hot topic that is trending is the special tr...


# 2. News Articles

We will now be scraping text data from inshorts, a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

    Business
    Sports
    Technology
    Entertainment

The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:

{
    'title': 'The article title',
    'content': 'The article content',
    'category': 'business' # for example
}

In [19]:
url = 'https://inshorts.com/en/read'
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [20]:
articles = []
dct = {}
dct['title'] = soup.select('.news-card')[0].select('.news-card-title')[0].span.text
dct['content'] = soup.select('.news-card')[0].select('.news-card-content')[0].div.text

In [2]:
def get_news_articles(categories=['business', 'sports', 
                                  'technology', 'entertainment'], 
                      use_cache=True):
    
    # establish a filename for the local csv
    filename = 'news_articles.csv'
    
    if use_cache:
        # check to see if a local copy already exists
        if os.path.exists(filename):
            print('Reading from local CSV...')
            # if so, return the local csv
            return pd.read_csv(filename)
        
    # otherwise, scrape the data from codeup.com
    print('Reading blog articles from inshorts.com...')

    articles = []

    for category in categories:

        url = f'https://inshorts.com/en/read/{category}'
        response = get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        for card in soup.select('.news-card'):
            dct = {}
            dct['title'] = card.select('.news-card-title')[0].span.text
            dct['author'] = card.select('.author')[0].text
            dct['content'] = card.select_one('.news-card-content').div.text
            dct['category'] = category
            articles.append(dct)
            
    articles = pd.DataFrame(articles)
    
    # cache local copy
    print('Writing to local CSV...')
    articles.to_csv(filename, index=False)
            
    return articles

In [5]:
get_news_articles()

Reading from local CSV...


,title,author,content,category
0,Rupee hits all-time low of 77.42 against US do...,Apaar Sharma,The Indian rupee fell to an all-time low of 77...,business
1,Bitcoin falls to the lowest level since Januar...,Pragya Swastik,"Bitcoin fell on Monday to as low as $33,266 in...",business
2,Made best possible decision: IndiGo on barring...,Pragya Swastik,IndiGo's CEO Ronojoy Dutta said the airline ma...,business
3,India's biggest IPO of LIC subscribed nearly 3...,Pragya Swastik,"LIC's IPO, India's biggest IPO which opened on...",business
4,"If I die under mysterious circumstances, nice ...",Ridham Gambhir,"Tesla CEO Elon Musk has tweeted, ""If I die und...",business
...,...,...,...,...
94,He's done right thing: Neetu on Ranbir not bei...,Kriti Kambiri,Actress Neetu Kapoor has said that her son Ran...,entertainment
95,Fred Savage fired from Wonder Years after alle...,Udit Gupta,Fred Savage has been dropped from the comedy s...,entertainment
96,Comedy is the nicest genre; it's difficult to ...,Mahima Kharbanda,Actress Samantha Prabhu said that she wants to...,entertainment
97,"People taunted me for not getting pregnant, tr...",Kriti Kambiri,Actress Sambhavna Seth has revealed that she h...,entertainment
